In [1]:
import tensorflow as tf
import tensorflow.keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from IPython.display import clear_output

import numpy as np
import os 

%run midi2text2midi.ipynb

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## Read data

#### Keep timesigs 4/4, 3/4, 2/4, 6/8

In [2]:
filename = '../dataset_text/miditokens_waitFix.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
miditokens_tempo_and_sig = [tokens.strip().split(' ') for tokens in miditokens]

miditokens = []
for song in miditokens_tempo_and_sig:
    sig = song[2]
    if sig in ['timesig:4/4', 'timesig:3/4', 'timesig:2/4', 'timesig:6/8']:
        miditokens.append(song)
print("Number of songs: {0}".format(len(miditokens)))

Number of songs: 550


## Tokenization

In [3]:
tokenizer = Tokenizer(oov_token='x') # token -> int
tokenizer.fit_on_texts(miditokens)

print("Number of unique tokens: {0}".format(len(tokenizer.word_index)))

# Example of token input -> int output
# sample = miditokens[0]
# print(tokenizer.texts_to_sequences([sample])[0]) 

# Get count of word in whole dataset
print(tokenizer.word_counts['wait:1.0'])

# Get class index of a token 
print(tokenizer.word_index['wait:1.0'])

# Get token of a class index
print(tokenizer.index_word[27])


# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)

# Get inverse map of tokenizer
intToNote = dict(map(reversed, tokenizer.word_index.items()))

Number of unique tokens: 310
13854
27
wait:1.0


# Generator variant

In [4]:
SEQ_LEN = 50
pathToOutput = "../dataset_text/seq2note_int.txt"

def writeGeneratorLinesToFile(pathToOutputFile):
    with open(pathToOutputFile, "a") as f:
        for song in midiTokensAsInt:
            for i in range(0, len(song) - SEQ_LEN, 1):
                song = [str(token) for token in song]
                f.write(' '.join(song[i:i + SEQ_LEN]) + ", " + (song[i + SEQ_LEN]) + '\n')
                
def getNumberOfLinesInGeneratorFile():
    LINES = 0
    for song in midiTokensAsInt:
        for idx in range(0, len(song) - SEQ_LEN, 1):
            LINES += 1
    return LINES

In [5]:
# https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366

LINES = getNumberOfLinesInGeneratorFile()

BATCH_SIZE = 512
VOCAB_SIZE = len(tokenizer.word_index)
steps = LINES // BATCH_SIZE

with open("../dataset_text/seq2note_int.txt", "r") as f:
    lines = f.readlines()

def batchGenerator(trainData, lines, steps, VOCAB_SIZE, LINES, SEQ_LEN=50, BATCH_SIZE=32):
    lastLine = 0
    while True:
        
        # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
        
        X_train = []
        y_train = []
        
        for idx in range(lastLine, min(lastLine + BATCH_SIZE, LINES), 1):
            sample = lines[idx].split(", ")
            X_train.append([int(i) for i in sample[0].split(" ")])
            y_train.append(int(sample[1]))

        X_train = to_categorical(X_train, num_classes=VOCAB_SIZE+1)
        y_train = to_categorical(y_train, num_classes=VOCAB_SIZE+1)
        
        yield (X_train, y_train)
        
        lastLine += BATCH_SIZE
        if lastLine > LINES:
            lastLine = 0

batchGen = batchGenerator(midiTokensAsInt, lines, steps, VOCAB_SIZE, LINES, SEQ_LEN, BATCH_SIZE)
#test = next(batchGen)

### Training

In [ ]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(SEQ_LEN, VOCAB_SIZE+1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE+1))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "../weights/weights-generator/weights-{epoch:02d}-{loss:.4f}.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

model.fit(batchGen, 
          workers=0,
          steps_per_epoch = steps,
          epochs=200,
          callbacks=callbacks_list,
          verbose=1)

In [ ]:
#model.save('../trained_models/' + str(input()))

In [6]:
model = tf.keras.models.load_model("../trained_models/LSTM256-generator2.h5")

### Function to generate subsequent tokens


In [7]:
def printLoading(tokensGenerated, toGenerate, generateUntilEnd):
    clear_output(wait=True)
    if (not generateUntilEnd):
        print("{0} / {1} generated".format(tokensGenerated, toGenerate))
        return
    print("{0} / {1} generated".format(tokensGenerated, "?"))

def generateSubsequentTokens(model, toGenerate = 500, startingInput = [], generateUntilEnd = False):
    # Range determines the number of tokens to predict
    
    keepGenerating = True
    tokensGenerated = 0
    
    slidingSequence = [startingInput.copy()]
    predictionOutput = [intToNote[i] for i in startingInput.copy()]
    
    while keepGenerating:
        # Convert to acceptable format for trained model
        prediction_input = to_categorical(slidingSequence, num_classes=VOCAB_SIZE+1)

        # Predict next token depending on the previous 50 tokens
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)

        # Check if previous tokens were "varied" enough, if not, choose a random prediction from the top 3 predictions
        if (len(np.unique(slidingSequence)) < 15):
            ind = np.argpartition(prediction[0], -3)[-3:]
            index = np.random.choice(ind)

        result = intToNote[index]

        predictionOutput.append(result)
        tokensGenerated += 1

        slidingSequence = np.append(slidingSequence, index)
        slidingSequence = [slidingSequence[1:len(slidingSequence)]]
        
        printLoading(tokensGenerated, toGenerate, generateUntilEnd)
        
        if (generateUntilEnd and result == "end") or (not generateUntilEnd and tokensGenerated == toGenerate):
            keepGenerating = False
            
    return predictionOutput

In [9]:
songTokens = tokenizer.texts_to_sequences([midi2text(open_midi("../testmidis/supermario.mid"))])[0][:50]
generatedTokens = generateSubsequentTokens(model, startingInput = songTokens, generateUntilEnd = False)
genmidistream = text2midi(generatedTokens)
genmidistream.show("midi")

500 / 500 generated


In [ ]:
genmidistream.write('midi', fp='../results/' + str(input()) + '.mid')